In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#            Machine Learning y Deep Learning
# ----------------------------------------------------------
#         Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#   Redes neuronales LSTM para la prediccion de BitCoin
# ==========================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go

In [ ]:
# ---------------------- Cargando datos ----------------------------------------
# Cargar el conjunto de datos:
url =  "https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/BTCUSD.csv"
# Cargar los datos en un DataFrame
data= pd.read_csv(url)
# Mostrar las primeras filas del DataFrame
print(data.head())

In [ ]:
# Cargar el conjunto de datos:
url =  "https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/ETHUSD.csv"
# Cargar los datos en un DataFrame
data_ETHUSD= pd.read_csv(url)
# Mostrar las primeras filas del DataFrame
print(data_ETHUSD.head())

In [ ]:
# Grafico candlestick Bitcoin
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])
fig.update_layout(
    title='Bitcoin Candlestick',
    yaxis_title='Precio USD',
    xaxis_title='Fecha'
)
fig.show()

In [ ]:
# Grafico candlestick Ethereum
fig = go.Figure(data=[go.Candlestick(x=data_ETHUSD['Fecha'],
                open=data_ETHUSD['Open'],
                high=data_ETHUSD['High'],
                low=data_ETHUSD['Low'],
                close=data_ETHUSD['Close'])])
fig.update_layout(
    title='Ethereum Candlestick',
    yaxis_title='Precio USD',
    xaxis_title='Fecha'
)
fig.show()

In [ ]:
# Preparar el objetivo (y)
y = data['Close'].values.reshape(-1, 1)

# Crear la matriz de características X como rezagos de y
def create_lagged_features(y, lag=1):
    X = np.zeros((len(y) - lag, lag))
    for i in range(lag):
        X[:, i] = y[(lag - i - 1):(len(y) - i - 1), 0]
    y_lagged = y[lag:]
    return X, y_lagged

# Usar rezagos para las características
lag = 60
X, y_lagged = create_lagged_features(y, lag)

# Normalizar los datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X = scaler_X.fit_transform(X)
y_lagged = scaler_y.fit_transform(y_lagged)

# Reshape para que sea compatible con LSTM (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Partición de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y_lagged, test_size=0.15, shuffle=False)

# Inicialización del modelo LSTM
model = Sequential()
# model.add(Input(shape=(X_train.shape[1], 1)))  # Definir la capa de entrada explícitamente
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
epochs = 100
model.fit(X_train, y_train, epochs=epochs)

# Predicción en el conjunto de prueba
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Desnormalizar para evaluar correctamente
y_train_actual = scaler_y.inverse_transform(y_train)
y_test_actual = scaler_y.inverse_transform(y_test)
y_pred_train_actual = scaler_y.inverse_transform(y_pred_train)
y_pred_test_actual = scaler_y.inverse_transform(y_pred_test)

# Evaluación del modelo
mae_train = mean_absolute_error(y_train_actual, y_pred_train_actual)
rmse_train = np.sqrt(mean_squared_error(y_train_actual, y_pred_train_actual))
mae_test = mean_absolute_error(y_test_actual, y_pred_test_actual)
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_pred_test_actual))

print(f"MAE (train): {mae_train}")
print(f"RMSE (train): {rmse_train}")
print(f"MAE (test): {mae_test}")
print(f"RMSE (test): {rmse_test}")

# Gráficos
plt.figure(figsize=(14, 7))

# Predicciones vs Realidad
plt.plot(y_test_actual, label='Valores observados')
plt.plot(y_pred_test_actual, label='Valores predichos')
plt.title('Comparación entre los precios reales y predichos')
plt.xlabel('Días')
plt.ylabel('Precio Bitcoin (USD)')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# ---------------------- Modelo RNA LSTM para Ethereum ----------------------------------------
# Preparar el objetivo (y)
y = data_ETHUSD['Close'].values.reshape(-1, 1)
# Crear la matriz de características X como rezagos de y
def create_lagged_features(y, lag=1):
    X = np.zeros((len(y) - lag, lag))
    for i in range(lag):
        X[:, i] = y[(lag - i - 1):(len(y) - i - 1), 0]
    y_lagged = y[lag:]
    return X, y_lagged

# Usar rezagos para las características
lag = 60
X, y_lagged = create_lagged_features(y, lag)

# Normalizar los datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X = scaler_X.fit_transform(X)
y_lagged = scaler_y.fit_transform(y_lagged)

# Reshape para que sea compatible con LSTM (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Partición de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y_lagged, test_size=0.15, shuffle=False)

# Inicialización del modelo LSTM
model = Sequential()
# model.add(Input(shape=(X_train.shape[1], 1)))  # Definir la capa de entrada explícitamente
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
epochs = 100
model.fit(X_train, y_train, epochs=epochs)

# Predicción en el conjunto de prueba
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Desnormalizar para evaluar correctamente
y_train_actual = scaler_y.inverse_transform(y_train)
y_test_actual = scaler_y.inverse_transform(y_test)
y_pred_train_actual = scaler_y.inverse_transform(y_pred_train)
y_pred_test_actual = scaler_y.inverse_transform(y_pred_test)

# Evaluación del modelo
mae_train = mean_absolute_error(y_train_actual, y_pred_train_actual)
rmse_train = np.sqrt(mean_squared_error(y_train_actual, y_pred_train_actual))
mae_test = mean_absolute_error(y_test_actual, y_pred_test_actual)
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_pred_test_actual))

print(f"MAE (train): {mae_train}")
print(f"RMSE (train): {rmse_train}")
print(f"MAE (test): {mae_test}")
print(f"RMSE (test): {rmse_test}")

# Gráficos
plt.figure(figsize=(14, 7))

# Predicciones vs Realidad
plt.plot(y_test_actual, label='Valores observados')
plt.plot(y_pred_test_actual, label='Valores predichos')
plt.title('Comparación entre los precios reales y predichos')
plt.xlabel('Días')
plt.ylabel('Precio Ethereum (USD)')
plt.legend()

plt.tight_layout()
plt.show()